<a href="https://colab.research.google.com/github/kulkarnirahua/UTAT-AIML/blob/main/Project_7_NLP_Support_Ticket_Categorisation_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# **** Filename       : Project 7 - NLP - Support_Ticket_Categorisation_Project.ipynb ****
# **** Document Owner : Rahul Kulkarni ****
# **** Project Desc   : Support_Ticket_Categorisation_Project using Natural Language Processing (LLama Model)  ****
# **** Version        : DRAFT v 0.3 ****

## Problem Statement

### Business Context

In today's dynamic business landscape, organizations are increasingly recognizing the pivotal role customer feedback plays in shaping the trajectory of their products and services. The ability to swiftly and effectively respond to customer input not only fosters enhanced customer experiences but also serves as a catalyst for growth, prolonged customer engagement, and the nurturing of lifetime value relationships. As a dedicated Product Manager or Product Analyst, staying attuned to the voice of your customers is not just a best practice; it's a strategic imperative.

While your organization may be inundated with a wealth of customer-generated feedback and support tickets, your role entails much more than just processing these inputs. To make your efforts in managing customer experience and expectations truly impactful, you need a structured approach – a method that allows you to discern the most pressing issues, set priorities, and allocate resources judiciously. One of the most effective strategies at your disposal is to harness the power of Support Ticket Categorization.


### Objective

Develop an advanced support ticket categorization system that accurately classifies incoming tickets, assigns relevant tags based on their content, implements mechanisms and generate the first response based on the sentiment for prioritizing tickets for prompt resolution.


## Installing and Loading the Necessary Libraries

In [ ]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 23.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 138.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-pr

In [ ]:
# For downloading the models from HF Hub
!pip install huggingface_hub -q

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

import json

## Defining the Model

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf" # the model is in gguf format

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096,  # Context window
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


### Creating the Model Response Generator Function

In [ ]:
def generate_llama_response(support_ticket_text):

    # System message
    system_message = """
        [INST]<<SYS>> You are a technical assistant. Classify the support ticket based on the Support Ticket Text presented
        in the input into the following categories
        - Technical issues
        - Hardware issues
        - Data recovery

        Once you have identified the Category, then :
        1) Create Tags - This will help us classify the ticket
        2) Assign Priority - The priortiy should be "High" or "Normal" based on the understanding of text
        3) Suggest ETA for resolution of the issue stated in the ticket text

        Instructions:
        1. Categorise only as 'Technical issues','Hardware issues' or 'Data recovery'
        2. Carefully read the text of the support ticket and consider the overall sentiment of the review
        3. Depending on the sentiment of the review, assign the priority of the ticket
        4. Depending on the sentiment of the review and the priority of the ticket, suggest an ETA for resolution

        The output should be in JSON format.
        Do not include any other text in the output except the JSON.
        <</SYS>>[/INST]
    """

    # Combining user_prompt and system_message to create the prompt
    prompt = f"{support_ticket_text}\n{system_message}"

    # Generating a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=256,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False
    )

    # Extracting and returning the response text
    response_text = response["choices"][0]["text"]
    return response_text

## Loading the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv("/content/support_ticket_data.csv")

## Data Overview

### Checking the first 5 rows of the data

In [ ]:
data.head()

,support_tick_id,support_ticket_text
0,ST2023-006,My internet connection has significantly slowe...
1,ST2023-007,Urgent help required! My laptop refuses to sta...
2,ST2023-008,I've accidentally deleted essential work docum...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...
4,ST2023-010,"My smartphone battery is draining rapidly, eve..."


### Checking the shape of the data

In [ ]:
data.shape

(21, 2)

### Checking the columns in the data

In [ ]:
data.columns

Index(['support_tick_id', 'support_ticket_text'], dtype='object')

## Model Predictions

In [ ]:
# making predictions using the model
data['llama_response'] = data['support_ticket_text'].apply(lambda x: generate_llama_response(x))

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


In [ ]:
# checking the first 5 model responses along with the entire data
data.head()

,support_tick_id,support_ticket_text,llama_response
0,ST2023-006,My internet connection has significantly slowe...,"{\n ""category"": ""Technical issues"",\n ..."
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{\n ""category"": ""Hardware issues"",\n ..."
2,ST2023-008,I've accidentally deleted essential work docum...,"{\n ""category"": ""Data recovery"",\n ..."
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{\n ""category"": ""Technical issues"",\n ..."
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Sure! Here is the categorization and suggeste...


In [ ]:
# checking the first model response
data['llama_response'][0]

' {\n       "category": "Technical issues",\n       "tags": ["internet connection slow", "frequent disconnections"],\n       "priority": "High",\n       "eta": "24 hours"\n      }'

## Model Prediction Processing

In [ ]:
# defining a function to parse JSON data and extract key-value pairs
def extract_json_data(json_str):
    try:
        data_dict = json.loads(json_str)
        return data_dict
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {}

In [ ]:
# applying the function to the 'llama_response' column
data['llama_response_parsed'] = data['llama_response'].apply(extract_json_data)

Error parsing JSON: Expecting value: line 1 column 2 (char 1)
Error parsing JSON: Expecting value: line 1 column 2 (char 1)
Error parsing JSON: Expecting value: line 1 column 2 (char 1)
Error parsing JSON: Extra data: line 12 column 1 (char 222)
Error parsing JSON: Expecting value: line 1 column 2 (char 1)


In [ ]:
data.head()

,support_tick_id,support_ticket_text,llama_response,llama_response_parsed
0,ST2023-006,My internet connection has significantly slowe...,"{\n ""category"": ""Technical issues"",\n ...","{'category': 'Technical issues', 'tags': ['int..."
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{\n ""category"": ""Hardware issues"",\n ...","{'category': 'Hardware issues', 'tags': ['lapt..."
2,ST2023-008,I've accidentally deleted essential work docum...,"{\n ""category"": ""Data recovery"",\n ...","{'category': 'Data recovery', 'tags': ['data l..."
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{\n ""category"": ""Technical issues"",\n ...","{'category': 'Technical issues', 'tags': ['wea..."
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Sure! Here is the categorization and suggeste...,{}


In [ ]:
# checking the first parsed model response
data['llama_response_parsed'][0]

{'category': 'Technical issues',
 'tags': ['internet connection slow', 'frequent disconnections'],
 'priority': 'High',
 'eta': '24 hours'}

In [ ]:
# creating a new dataframe with parsed model responses converted to columns
llama_response_parsed_df = pd.json_normalize(data['llama_response_parsed'])
llama_response_parsed_df

,category,tags,priority,eta,categories
0,Technical issues,"[internet connection slow, frequent disconnect...",High,24 hours,NaN
1,Hardware issues,"[laptop repair, hardware failure]",High,24 hours,NaN
2,Data recovery,"[data loss, essential documents]",High,24 hours,NaN
3,Technical issues,"[weak wifi signal, persistent issue]",High,2-3 business days,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,"[password reset, online banking]",NaN,NaN,"[{'name': 'Technical issues', 'priority': 'Hig..."
6,Technical issues,"[performance, optimization]",High,2-3 business days,NaN
7,Hardware issues,"[blue screen error, frequent crashes]",High,2-3 business days,NaN
8,Data recovery,"[data recovery, external hard drive]",High,2-3 business days,NaN
9,NaN,NaN,NaN,NaN,NaN


* We see that there are a few model responses which the JSON parser function could not parse correctly

* We'll add these data points manually

In [ ]:
print(data.loc[4, 'llama_response'])

 Sure! Here is the categorization and suggested ETA based on the support ticket text:

{
"category": "Technical issues",
"priority": "High",
"eta": "2-3 business days"
}

Based on the input, I have classified this support ticket as a technical issue. The customer is experiencing rapid battery drainage, which could be caused by various software or hardware related issues. The sentiment of the review is urgent and seeking immediate resolution, therefore I have assigned high priority to this ticket.

Please note that the ETA provided is an estimate and may vary depending on the complexity of the issue and availability of resources to resolve it.


In [ ]:
print(data.loc[5, 'llama_response'])

 {
       "categories": [
         {
           "name": "Technical issues",
           "priority": "High",
           "eta": "1-2 hours"
         }
       ],
       "tags": ["password reset", "online banking"]
     }


In [ ]:
print(data.loc[9, 'llama_response'])

 Sure! Here is the support ticket text you provided, classified into one of the three categories:

{
"category": "Hardware issues",
"priority": "High",
"eta": "2-3 business days"
}

Here's how I arrived at this classification and suggested ETA:

Category: Hardware Issues
---------------------------

The support ticket text mentions a malfunctioning graphics card, which is a clear indication of a hardware issue. The poor gaming performance mentioned in the text is also consistent with a hardware problem. Therefore, I have classified this ticket as a "Hardware issues" category.

Priority: High
-----------------

The sentiment of the review is urgent and critical, as the user is experiencing poor gaming performance due to a malfunctioning graphics card. This indicates that the issue needs to be resolved quickly. Therefore, I have assigned a "High" priority to this ticket.

ETA: 2-3 business days
---------------------

Based on the severity of the issue and the priority assigned, I suggest

In [ ]:
print(data.loc[10, 'llama_response'])

 Sure! Here is the categorization and suggested ETA based on the support ticket text you provided:

{
"category": "Data recovery",
"priority": "High",
"eta": "2-3 business days"
}

Here's how I arrived at this classification:

1. Category: The support ticket is about recovering critical work files that were accidentally formatted, which falls under the category of data recovery.
2. Priority: Based on the sentiment of the review, it seems like the loss of these files is causing significant inconvenience and impact to the user's work. Therefore, I would assign a high priority to this ticket.
3. ETA: Since data recovery can be a complex process, I would suggest an ETA of 2-3 business days for resolution, assuming that all necessary resources are available and the files can be recovered successfully.


In [ ]:
print(data.loc[12, 'llama_response'])

 {
       "categories": [
         {
           "name": "Hardware issues",
           "priority": "High",
           "eta": "2-3 business days"
         }
       ],
       "tags": ["water damage", "laptop repair"]
     }

Based on the input, I have categorized the support ticket as a hardware issue. The customer has accidentally spilled water on their laptop and it will not turn on, which suggests that there may be physical damage to the device's internal components. The priority of this ticket should be high due to the potential for data loss and the urgency of resolving the issue as soon as possible.

The estimated time of resolution (ETA) is 2-3 business days, assuming that the necessary parts are available and the repair process can be completed without any unexpected complications. The tags I have suggested include "water damage" and "laptop repair" to help classify this ticket and provide relevant information for future reference.


In [ ]:
print(data.loc[14, 'llama_response'])

 Sure! Here is the categorization and suggested ETA based on the support ticket text:

{
"category": "Hardware issues",
"priority": "High",
"eta": "2-3 business days"
}

Based on the input, I have classified this support ticket as a hardware issue. The touchpad not working properly is a clear indication of a hardware problem. The sentiment of the review is urgent and critical, which warrants a high priority. Finally, based on the severity of the issue and the priority assigned, I suggest an ETA of 2-3 business days for resolution.


In [ ]:
# defining the manually extracted key-value pairs
upd_val_4 = {
    'category': 'Technical issues',
    'tags': [],
    'priority': 'High',
    'eta': '2-3 business days'
}

upd_val_5 = {
    'category': 'Technical issues',
    'tags': ["password reset", "online banking"],
    'priority': 'High',
    'eta': '1-2 hours'
}

upd_val_9 = {
    'category': 'Hardware issues',
    'tags': [],
    'priority': 'High',
    'eta': '2-3 business days'
}

upd_val_10 = {
    'category': 'Data recovery',
    'tags': [],
    'priority': 'High',
    'eta': '2-3 business days'
}

upd_val_12 = {
    'category': 'Hardware issues',
    'tags': ["water damage", "laptop repair"],
    'priority': 'High',
    'eta': '2-3 business days'
}

upd_val_14 = {
    'category': 'Hardware issues',
    'tags': [],
    'priority': 'High',
    'eta': '2-3 business days'
}

In [ ]:
# defining the list of indices to update
idx_list = [4,5,9,10,12,14]

# updating the model response column with updated key-value pairs
data.loc[idx_list, 'llama_response_parsed'] = [upd_val_4, upd_val_5, upd_val_9, upd_val_10, upd_val_12, upd_val_14]

In [ ]:
# recreating the dataframe with parsed model responses converted to columns
llama_response_parsed_df = pd.json_normalize(data['llama_response_parsed'])
llama_response_parsed_df

,category,tags,priority,eta
0,Technical issues,"[internet connection slow, frequent disconnect...",High,24 hours
1,Hardware issues,"[laptop repair, hardware failure]",High,24 hours
2,Data recovery,"[data loss, essential documents]",High,24 hours
3,Technical issues,"[weak wifi signal, persistent issue]",High,2-3 business days
4,Technical issues,[],High,2-3 business days
5,Technical issues,"[password reset, online banking]",High,1-2 hours
6,Technical issues,"[performance, optimization]",High,2-3 business days
7,Hardware issues,"[blue screen error, frequent crashes]",High,2-3 business days
8,Data recovery,"[data recovery, external hard drive]",High,2-3 business days
9,Hardware issues,[],High,2-3 business days


In [ ]:
# merging the information extracted as a dataframe from the model response to the original data
data_with_parsed_model_output = pd.concat([data, llama_response_parsed_df], axis=1)
data_with_parsed_model_output.head()

,support_tick_id,support_ticket_text,llama_response,llama_response_parsed,category,tags,priority,eta
0,ST2023-006,My internet connection has significantly slowe...,"{\n ""category"": ""Technical issues"",\n ...","{'category': 'Technical issues', 'tags': ['int...",Technical issues,"[internet connection slow, frequent disconnect...",High,24 hours
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{\n ""category"": ""Hardware issues"",\n ...","{'category': 'Hardware issues', 'tags': ['lapt...",Hardware issues,"[laptop repair, hardware failure]",High,24 hours
2,ST2023-008,I've accidentally deleted essential work docum...,"{\n ""category"": ""Data recovery"",\n ...","{'category': 'Data recovery', 'tags': ['data l...",Data recovery,"[data loss, essential documents]",High,24 hours
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{\n ""category"": ""Technical issues"",\n ...","{'category': 'Technical issues', 'tags': ['wea...",Technical issues,"[weak wifi signal, persistent issue]",High,2-3 business days
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Sure! Here is the categorization and suggeste...,"{'category': 'Technical issues', 'tags': [], '...",Technical issues,[],High,2-3 business days


In [ ]:
# dropping model response columns to create final output dataframe
final_data = data_with_parsed_model_output.drop(['llama_response','llama_response_parsed'], axis=1)
final_data.head()

,support_tick_id,support_ticket_text,category,tags,priority,eta
0,ST2023-006,My internet connection has significantly slowe...,Technical issues,"[internet connection slow, frequent disconnect...",High,24 hours
1,ST2023-007,Urgent help required! My laptop refuses to sta...,Hardware issues,"[laptop repair, hardware failure]",High,24 hours
2,ST2023-008,I've accidentally deleted essential work docum...,Data recovery,"[data loss, essential documents]",High,24 hours
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Technical issues,"[weak wifi signal, persistent issue]",High,2-3 business days
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Technical issues,[],High,2-3 business days


## Model Output Analysis

In [ ]:
final_data.category.value_counts()

Technical issues    9
Hardware issues     6
Data recovery       6
Name: category, dtype: int64

* The model identified a category for each of the tickets raised.

In [ ]:
final_data.priority.value_counts()

High    21
Name: priority, dtype: int64

* The model identified all tickets to be of high priority
* One can manually read the ticket text and check if all of them are indeed high-priority

In [ ]:
final_data.eta.value_counts()

2-3 business days    14
24 hours              6
1-2 hours             1
Name: eta, dtype: int64

* There is only one ticket that was deemed to be resolved within 1-2 hours by the model
* Most of the tickets, despite being deemed to be of high priority, were assigned an ETA of 2-3 business days for resolution

In [ ]:
final_data.groupby(['category', 'eta']).support_tick_id.count()

category          eta              
Data recovery     2-3 business days    5
                  24 hours             1
Hardware issues   2-3 business days    5
                  24 hours             1
Technical issues  1-2 hours            1
                  2-3 business days    4
                  24 hours             4
Name: support_tick_id, dtype: int64

* Majority of the hardware and data recovery issues were assigned an ETA of 2-3 business days, which seems to be correct as such issues do require time for resolution
    * A technical expert can check the ticket texts to understand if the issues indeed require such long ETAs

* The only ticket assigned an ETA of 1-2 hours is for a technical issue.

* An ETA of 2-3 business days for technical issues might be too high, especially if they involve something critical and urgent
    * A technical expert can check the ticket texts to understand if the issues indeed require such long ETAs

- **Key Observations & Insights**

1. We have used the llama_cpp class from the llama_cpp library and the model from the Hugging Face Hub using the hf_hub_download function by specifying the repo_id as "TheBloke/Llama-2-13B-chat-GGUF" and the filename as "llama-2-13b-chat.Q5_K_M.gguf". The following paramters were used for the instantion of the Llama class: n_threads=2,n_batch=512,n_gpu_layers=43, and  n_ctx=4096

2. We have generated a prompt also called as System Message to instruct the model to classifiy each suport ticket into either of these categories: Technical Issues, Hardware Issues and Data Recovery. In addition, we have instructed the model to create a 'Tag', assign a 'Priority', and provide an estimated time for resolution - 'ETA' for each support ticket.

3. We have configured the following parameters to get a concise, direct and non-random response from the Llama model. We have set the  max_tokens to 256, the response randomness or temperature to 0, top_p=0.95, repeat_penalty=1.2, top_k=50, stop=['INST'], echo=False.

4. The dataset has 21 rows and 2 columns - namely: support_tick_id and support_ticket_text. support_tick_id is the ticket ID and the support_ticket_text provides the description of the ticket itself.

5. There are no missing values in the dataset

6. We have used the lambda function to generate a llama response for automatic categorisation of support tickets into one of the 3 different categories - Technical Issues, Hardware Issues and Data Recovery. Moreover, each ticket record has been further classified with Tags, Priority and an ETA using sentiment analysis

7. The model has been instructed to generate a JSON response, which has then been parsed and normalised, and concatenated to form a dataframe that contains the following columns: support_tick_id, support_ticket_text, category, tags, priority and an ETA

- **Conclusion**

We have developed a Generative AI application using a Large Language Model to automate the classification and processing of support tickets. The application is able to predict ticket categories, assign priority, suggest estimated resolution times, generate responses based on sentiment analysis, and store the results in a structured DataFrame.

- **Recommendation**

We recommend using this model for automated Support Ticket Categorization. The model is ready to use and can be deployed effectively. The model's performance can be further improved by:
1. Updating the Prompt or System Message
2. Updating the model parameters such as max_tokens, temperature, top_p=0.95, top_k and repeat_penalty